In [ ]:
%matplotlib inline

import ffai.ai
import gym
import matplotlib.pyplot as plt
import numpy as np

from ffai.ai.env import FFAIEnv

plt.rcParams['figure.dpi'] = 90
plt.rcParams['font.family'] = "Cochin"
plt.rcParams['font.size'] = 12


seed = 1024
np.random.seed(seed)
random_state = np.random.RandomState(seed)
    
def ffai_random_action_to_setup(environment, seed=None):
#    environment.seed(seed)
    observation = environment.reset()
    print(f'{environment.get_seed()} =?= {seed}')

    reward = 0
    is_terminal = 0
    in_setup = lambda procedure: procedure[0:7].sum() > 0
    procedure = observation['procedure']

    while in_setup(procedure):
        random_action = get_random_action(environment, random_state)
        observation, reward, is_terminal, _ = environment.step(random_action)
        procedure = observation['procedure']

    return observation, reward, is_terminal, None


def get_random_action(envrironment, random_state):
    action_type = random_state.choice(environment.available_action_types())
    positions = envrironment.available_positions(action_type)
    action_position = random_state.choice(positions) if len(positions) > 0 else None

    random_action = {
        'action-type': action_type,
        'x': action_position.x if action_position is not None else None,
        'y': action_position.y if action_position is not None else None,
    }
    return random_action


environment = gym.make('FFAI-v1')
observation, reward, is_terminal, _ = ffai_random_action_to_setup(environment, seed=seed)

# Exploring the State Space

In [ ]:
nrows, ncols = 7, 4

fig, axs = plt.subplots(nrows, ncols)
features = iter(observation['board'].items())

for i in range(nrows):
    for j in range(ncols):
        feature_name, feature = next(features)
        
        axs[i,j].matshow(feature, cmap='Blues')
        axs[i,j].set_xticks([])
        axs[i,j].set_yticks([])
        axs[i,j].set_title(feature_name)
        
fig.set_figwidth(8)
fig.set_figheight(9.8)
fig.tight_layout()
fig.savefig('/Users/cswiercz/Desktop/lrl/features.png', transparent=True)

In [ ]:
def plot_board(observation, show_grid=True):

    fig, ax = plt.subplots(1, 1)

    active_players = observation['board']['active players']
    own_players = observation['board']['own players']
    opp_players = observation['board']['opp players']
    standing_players = observation['board']['standing players']
    roll_probabilities = observation['board']['roll probabilities']
    
    downed_players = own_players + opp_players - standing_players
    players = (own_players + 0.5*opp_players) - 0.3*downed_players

    mat = roll_probabilities - players
    ax.matshow(mat, cmap='RdBu', vmin=-1, vmax=1)

    nrows, ncols = active_players.shape
    ax.set_xticklabels(np.arange(ncols))
    ax.set_yticklabels(np.arange(nrows))
    
    if show_grid:
        ax.set_xticks(np.arange(ncols) + 0.5)
        ax.set_yticks(np.arange(nrows) + 0.5)
        ax.set_xticklabels(np.arange(ncols), fontdict={'horizontalalignment': 'right'})
        ax.set_yticklabels(np.arange(nrows), fontdict={'verticalalignment': 'bottom'})
    else:
        ax.set_xticks([])
        ax.set_yticks([])

    ax.grid(alpha=0.4)
    fig.tight_layout()
    fig.set_figwidth(9)
    ax.set_aspect(1.0)
    return fig

fig = plot_board(observation)

In [ ]:
def show_procedure(environment):
    procedure_index = np.where(observation['procedure'] == 1)[0][0]
    procedure_list = [
        'StartGame', 'CoinTossFlip', 'CoinTossKickReceive', 'Setup', 'PlaceBall', 'HighKick',
        'Touchback', 'Turn', 'PlayerAction', 'Block', 'Push', 'FollowUp', 'Apothecary',
        'PassAction', 'Catch', 'Interception', 'GFI', 'Dodge', 'Pickup']
    procedure = procedure_list[procedure_index]
    
    print(f'\nCurrent procedure: {procedure} (index = {procedure_index})')

show_procedure(environment)
observation['state']

# Exploring the Action Space

In [ ]:
from ffai.core.table import ActionType
from operator import itemgetter 


def show_available_actions(environment):
    action_types = list(environment.available_action_types())
    if len(action_types) > 1:
        actions = itemgetter(*action_types)(environment.actions)
    else:
        actions = [environment.actions[action_types[0]]]
    
    for i, action_type in enumerate(action_types):
        print(f'{actions[i]} (id = {action_type})')
        
        available_positions = environment.available_positions(action_type)
        for position in available_positions:
            print(f'\t{position.__dict__}')
    
    return action_types

def show_procedure(environment):
    procedure_index = np.where(observation['procedure'] == 1)[0][0]
    procedure_list = [
        'StartGame', 'CoinTossFlip', 'CoinTossKickReceive', 'Setup', 'PlaceBall', 'HighKick',
        'Touchback', 'Turn', 'PlayerAction', 'Block', 'Push', 'FollowUp', 'Apothecary',
        'PassAction', 'Catch', 'Interception', 'GFI', 'Dodge', 'Pickup']
    procedure = procedure_list[procedure_index]
    
    print(f'\nCurrent procedure: {procedure} (index = {procedure_index})')


fig = plot_board(observation)
fig.savefig('/Users/cswiercz/Desktop/lrl/example_simulation_1.png', transparent=True)

show_available_actions(environment)
show_procedure(environment)

In [ ]:
action = {
    'action-type': 27, # START_MOVE
    'x': 14,
    'y': 11,
}
observation, reward, is_terminal, _ = environment.step(action)
fig = plot_board(observation)

show_available_actions(environment)
show_procedure(environment)

fig.savefig('/Users/cswiercz/Desktop/lrl/roll_probabilities.png', transparent=True)
fig.savefig('/Users/cswiercz/Desktop/lrl/example_simulation_2.png', transparent=True)

In [ ]:
action = {
    'action-type': 22,  # MOVE
    'x': 13,
    'y': 12,
}
observation, reward, is_terminal, _ = environment.step(action)
fig = plot_board(observation)
fig.savefig('/Users/cswiercz/Desktop/lrl/example_simulation_3.png', transparent=True)

show_available_actions(environment)
show_procedure(environment)

In [ ]:
#plt.matshow(observation['board']['standing players'], cmap='binary')
action = {
    'action-type': 6, # USE_REROLL
    'x': -1,
    'y': -1,
}
observation, reward, is_terminal, _ = environment.step(action)
fig = plot_board(observation)
fig.savefig('/Users/cswiercz/Desktop/lrl/example_simulation_4.png', transparent=True)

show_available_actions(environment)
show_procedure(environment)

In [ ]:
action = {
    'action-type': 28, # START_BLOCK
    'x': 14,
    'y': 5,
}
observation, reward, is_terminal, _ = environment.step(action)
fig = plot_board(observation)
fig.savefig('/Users/cswiercz/Desktop/lrl/example_simulation_5.png', transparent=True)

show_available_actions(environment)
show_procedure(environment)

In [ ]:
action = {
    'action-type': 23, # BLOCK
    'x': 13,
    'y': 6,
}
observation, reward, is_terminal, _ = environment.step(action)
fig = plot_board(observation)

show_available_actions(environment)
show_procedure(environment)

In [ ]:
action = {
    'action-type': 13, # SELECT_DEFENDER_STUMBLES
    'x': -1,
    'y': -1,
}
observation, reward, is_terminal, _ = environment.step(action)
fig = plot_board(observation)

show_available_actions(environment)
show_procedure(environment)

In [ ]:
action = {
    'action-type': 18, # PUSH
    'x': 12,
    'y': 6,
}
observation, reward, is_terminal, _ = environment.step(action)
fig = plot_board(observation)
fig.savefig('/Users/cswiercz/Desktop/lrl/example_simulation_6.png', transparent=True)

show_available_actions(environment)
show_procedure(environment)

In [ ]:
action = {
    'action-type': 19, # FOLLOW_UP
    'x': 13,
    'y': 6,
}
observation, reward, is_terminal, _ = environment.step(action)
fig = plot_board(observation)
fig.savefig('/Users/cswiercz/Desktop/lrl/example_simulation_7.png', transparent=True)

show_available_actions(environment)
show_procedure(environment)